In [ ]:
#載入套件

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from time import sleep
from bs4 import BeautifulSoup as bs
import json
import os
from selenium.webdriver.common.keys import Keys
import requests as req
import openpyxl
from fake_useragent import UserAgent
ua = UserAgent(use_external_data=True)
import random
import jieba
import operator
from collections import Counter
from PIL import Image # 圖片轉array陣列
import matplotlib.pyplot as plt

!pip install wordcloud
from wordcloud import WordCloud # 文字雲
import pandas
import numpy

In [ ]:
#設定條件
#user = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
my_options = webdriver.ChromeOptions()
my_options.add_argument("--start-maximized")
my_options.add_argument("--incognito")
my_options.add_argument("--disable-popup-blocking")
my_options.add_argument("--disable-notifications")
my_options.add_argument("--lang=zh-TW")
#my_options.add_argument('--user-agent=%s' % user)


#service = Service(executable_path = "‪C:\\Users\\user\\Desktop\\python_monographic\\chromedriver.exe")

driver = webdriver.Chrome(options = my_options,service = Service(ChromeDriverManager().install()))
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
  "source": """
    Object.defineProperty(navigator, 'webdriver', {
      get: () => undefined
    })
  """
})

driver.get("https://www.google.com/travel/things-to-do?dest_src=ut&tcfs&ved=2ahUKEwjWs9eQ_J__AhUgRw8CHbAzBHsQ36MCegQIAhAF&ictx=3&hl=zh-Hant-TW&gl=tw&g2lb=2502548%2C2503771%2C2503781%2C4258168%2C4284970%2C4306835%2C4429192%2C4515404%2C4757164%2C4778035%2C4814050%2C4850738%2C4864715%2C4867855%2C4874190%2C4886480%2C4893075%2C4920132%2C4924070%2C4965990%2C4985712%2C4990494%2C72248281%2C72249293%2C72253158%2C72256471%2C72271034%2C72271797%2C72272556%2C72273215")
# https://www.tripadvisor.com.tw/

In [ ]:
import re
def data_clean(text):
    # 清洗excel中的非法字符，都是不常见的不可显示字符，例如退格，响铃等
    ILLEGAL_CHARACTERS_RE = re.compile(r'[\000-\010]|[\013-\014]|[\016-\037]')
    text = ILLEGAL_CHARACTERS_RE.sub(r'', text)
    return text

In [ ]:
# Part 1  蒐集景點資訊

In [ ]:
#網頁搜尋
def search_place():
    try:
        #找到搜尋框
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="yDmH0d"]/c-wiz[2]/div/div[2]/div/c-wiz/div[1]/div/div[1]/div/div[2]/div/div/div/div[1]/div/div/div[2]/input')))
        search_line = driver.find_element(By.XPATH,'//*[@id="yDmH0d"]/c-wiz[2]/div/div[2]/div/c-wiz/div[1]/div/div[1]/div/div[2]/div/div/div/div[1]/div/div/div[2]/input')

        #輸入關鍵字
        search_line.send_keys("台北")

        #送出
        webdriver.ActionChains(driver).send_keys(Keys.ENTER).perform()

        #睡一下再送出一次
        sleep(1)
        webdriver.ActionChains(driver).send_keys(Keys.ENTER).perform()

        #睡一下等網頁跑出來
        sleep(5)

        #找到"顯示所有結果的按鈕"，然後點擊它
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="yDmH0d"]/c-wiz[2]/div/div[2]/div/c-wiz/div/div/div[1]/div[2]/c-wiz/div/div[2]/div[3]/div/div/button/span')))
        show_more_data = driver.find_element(By.XPATH,'//*[@id="yDmH0d"]/c-wiz[2]/div/div[2]/div/c-wiz/div/div/div[1]/div[2]/c-wiz/div/div[2]/div[3]/div/div/button/span')
        show_more_data.click()

        #睡一下等網頁跑出來
        sleep(5)

    except TimeoutException:
        print("time over")

In [ ]:
#抓出所有搜尋結果的: 景點名稱, google評分, google評論數, 地點簡述, 評論連結
#因為景點的地址跟具體的評論不會直接顯示，所以要蒐集評論連結，之後再另外連上連結抓出地址跟評論

def get_all_place():
    #先抓出所有欄位
    try:
        #找出每個景點欄位
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div[class="NnEw9 OBk50c T1Yjbc"]')))
        all_place = driver.find_elements(By.CSS_SELECTOR,'div[class="NnEw9 OBk50c T1Yjbc"]')

    except TimeoutException:
        print("time over")


    #再設置一個裝資訊的list
    place_list = []

    #用for迴圈把每個欄位點開並蒐集資訊
    for index,item_data in enumerate(all_place):
        print(f"<<<<< 第{index+1}筆 >>>>>")

        #先抓出補充說明
        place_addition = item_data.find_element(By.CSS_SELECTOR,'div[class="GwjAi "] div[class="nFoFM"]').text

        #點開資訊欄
        item_data.click()

        sleep(1)

        #抓地名
        try:
            WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div[jsname="BEZjkb"] div[class="tWJQl"] div[class="m3JYm UjZ9Nc"] div[class="h2MAYb Zkzxcb"] h2')))
            place_name = driver.find_element(By.CSS_SELECTOR,'div[jsname="BEZjkb"] div[class="tWJQl"] div[class="m3JYm UjZ9Nc"] div[class="h2MAYb Zkzxcb"] h2').text
            print(place_name)
        except TimeoutException:
            print("time over")

        #抓評分
        try:
            WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div[jsname="BEZjkb"] div[class="tWJQl"] div[class="m3JYm UjZ9Nc"] div[class="h2MAYb PrbLub"]  div[class="U675cf"]  div[class="iDqPh BgYkof"]')))
            place_score = driver.find_element(By.CSS_SELECTOR,'div[jsname="BEZjkb"] div[class="tWJQl"] div[class="m3JYm UjZ9Nc"] div[class="h2MAYb PrbLub"]  div[class="U675cf"]  div[class="iDqPh BgYkof"]').text

            print(place_score)
        except TimeoutException:
            place_score = "null"
            print("time over")

        #抓評論數
        try:
            WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div[jsname="BEZjkb"] div[class="tWJQl"] div[class="U675cf sSHqwe"] a')))
            all_comment = driver.find_element(By.CSS_SELECTOR,'div[jsname="BEZjkb"] div[class="tWJQl"] div[class="U675cf sSHqwe"] a').text
            all_comment = all_comment.replace(",","")
            all_comment = all_comment.replace(" 則評論","")
            print(all_comment)
        except TimeoutException:
            all_comment = "null"
            print("time over")

        #抓地點說明
        print(place_addition)

        #搜索資訊欄裡面的評論連結
        try:
            WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div[jsname="BEZjkb"] div[class="tWJQl"] div[class="U675cf sSHqwe"] a')))
            comment_href = driver.find_element(By.CSS_SELECTOR,'div[jsname="BEZjkb"] div[class="tWJQl"] div[class="U675cf sSHqwe"] a').get_attribute("href")
            print(comment_href)
        except TimeoutException:
            comment_href = "null"
            print("time over")

        print("="*50)

        #做一個字典方便檢索
        data_dic = {"景點名稱":place_name,"google評分":place_score,"google評論數":all_comment,"地點簡述":place_addition,"評論連結":comment_href}

        #再放進list
        place_list.append(data_dic)

    #print(place_list)
    
    return place_list

In [ ]:
#把place_list每10筆資料分成一組，目的是為了分開執行抓取更詳細資料時被google的"我不是機器人"搞

def group_place_list(place_list):
    #先設一個總集合
    all_group_place = []
    
    #再設一個子集合
    small_group_place_list = []
    
    for item in place_list:
        if len(small_group_place_list) < 10:
            small_group_place_list.append(item)
            
        else:
            all_group_place.append(small_group_place_list)
            small_group_place_list = []

    else:
        all_group_place.append(small_group_place_list)
        small_group_place_list = []
            
    print(all_group_place)       
    return all_group_place

In [ ]:
#做一個控制現在要抓all_group_place裡面哪個小組資訊的func

def get_what_group():
    #要先知道裡面分成了幾組
    how_many_group = len(all_group_place)
    print("總共有:",how_many_group,"組")
    
    get_group = int(input("請選擇要搜索第幾組:"))
    
    return get_group

In [ ]:
#把group_place_list內的每個小組抓出來分別搜索裡面的地址

def get_place_address(all_group_place,get_group): 
    time_sleep = random.randint(5,30)
    
    get_group = get_group - 1

    for item in all_group_place[get_group]:

        #抓出存有連結的key-value
        item_href = item["評論連結"]
        print(item_href)

        #如果沒有連結就跳過
        if item_href != "null":
            #有連結就連上
            try:
                driver.get(item_href)

                #睡一下等網頁跑
                sleep(time_sleep)

            except:
                driver.refresh()
                sleep(time_sleep)

            finally:
                driver.refresh()

            sleep(20)

            try:
                #抓出地址的路徑
                WebDriverWait(driver,30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div[jsname="Sx9Kwc"] div[class="VFlF2c review-dialog"] div[class="Zd3Ymd review-dialog-body"] div[class="VUGnzb"] div[class="T6pBCe"]')))
                get_place = driver.find_element(By.CSS_SELECTOR,'div[jsname="Sx9Kwc"] div[class="VFlF2c review-dialog"] div[class="Zd3Ymd review-dialog-body"] div[class="VUGnzb"] div[class="T6pBCe"]').text
                print(get_place)
                
                #把市跟區切開
                city = get_place[:3]
                district = get_place[3:6]

                #新增地址到字典內
                item["地址"] = get_place
                item["縣市"] = city
                item["鄉鎮市區"] = district

            except TimeoutException:
                item["地址"] = "null"
                item["縣市"] = "null"
                item["鄉鎮市區"] = "null"
                print("time over")

        else:
            item["地址"] = "null"
            item["縣市"] = "null"
            item["鄉鎮市區"] = "null"


        sleep(time_sleep)


        print(item)

        print("="*50)

        sleep(time_sleep)
    
    #把小組丟出來
    new_group_place = all_group_place[get_group]    #由dict組成的list
    return new_group_place


In [ ]:
# ======================= Part 1 總執行 ===========================

In [ ]:
#將每個小組的完整資訊再統整起來
#開啟網頁  >>> 抓初步景點資訊  >>> 製成小組list

search_place()

place_list = get_all_place()

all_group_place = group_place_list(place_list)

final_place_data = []

In [ ]:
#從這邊控制抓取每一組景點(輸入要選哪一組)

#控制要爬哪一組  >>>  將每組爬完新增的資訊再裝起來

get_group = get_what_group()

new_group_place = get_place_address(all_group_place,get_group)

final_place_data.append(new_group_place)    #巨大的list,裡面由每組的list構成,每組的list裡面又是由dict構成 : [[{},{}],[{},{}],[]]

In [ ]:
print(final_place_data)

In [ ]:
#將最後集合完的list匯出成excel

#把抓出來的東西寫入excel
wb = openpyxl.Workbook()
wb.sheetnames
ws = wb.active
ws.append(["景點名稱","google評分","google評論數","地點簡述","評論連結","地址","縣市","鄉鎮市區"])
place_list = []

for index,y in enumerate(final_place_data):    #拆掉第一層list  >>>  y 是代表每個小組的list
    print(f"<<<<< 第{index+1}組 >>>>>\n")
    for z in y:    #拆掉第二層list  >>>  z 是代表每個小組內的景點資訊的 dict
    
        a = z["景點名稱"]
        
        try:
            b = float(z["google評分"])
            c = int(z["google評論數"])
        except:
            b = z["google評分"]
            c = z["google評論數"]
    
        d = z["地點簡述"]
        e = z["評論連結"]
        f = z["地址"]
        g = z["縣市"]
        h = z["鄉鎮市區"]
    
        ws.append([a,b,c,d,e,f,g,h])    
    print("="*50)      
    
wb.save("./place_data_0607.xlsx")


In [ ]:
# ======================= Part 1 over ===========================

In [ ]:
# Part 2

In [ ]:
#將評論選至最新

def set_newest():
    try:
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div[class="ABhRFe"] g-scrolling-carousel[class="R8LtDd"] div[jsname="haAclf"] div[jsname="s2gQvd"] div[data-sort-id="newestFirst"]')))
        new_comment = driver.find_element(By.CSS_SELECTOR,'div[class="ABhRFe"] g-scrolling-carousel[class="R8LtDd"] div[jsname="haAclf"] div[jsname="s2gQvd"] div[data-sort-id="newestFirst"]')
        new_comment.click()

    except TimeoutException:
        print('time over')

In [ ]:
#建立讓內嵌視窗滾動的function

def scroll():
    innerheight = 0
    offset = 0
    count = 1
    limit = 3
    
    while count <= limit:
        time_sleep = random.randint(4,7)
        
        #offset = "document.getElementsByClassName('review-dialog-list')[0].scrollTop = 10000"
        #driver.execute_script(offset)
        
        offset = offset + 10000

        driver.execute_script(f'''document.getElementsByClassName('review-dialog-list')[0].scrollTo({{top:{offset},behavior:'smooth'}});''')
        
        innerheight = driver.execute_script("return document.getElementsByClassName('review-dialog-list')[0].scrollHeight;")
        #print(innerheight)
        
        sleep(time_sleep)
        
        if offset == innerheight:
            count += 1
            
        if offset > 300000:
            break

In [ ]:
#用原始碼抓出所有評論，再用bs解析抓出所有評論跟時間
#因為selenium抓不出來

def get_all_comment():
    #抓出原始碼
    html = driver.page_source

    #丟bs
    soup = bs(html,'lxml')

    #抓出最小單位資訊集合
    a = soup.select('div[class="jxjCjc"] div[class="gQfZge"] div[style="vertical-align:top"]')

    #用for迴圈抓出每一則評論

    #先設一個list裝資訊
    comment_list = []

    for i in a:
        #時間
        time_comment = i.select_one('div[class="PuaHbe"] span[class="dehysf lTi8oc"]').text
        print(time_comment)

        #評分
        get_score = i.select_one('div[class="PuaHbe"] span[class="lTi8oc z3HNkc"]')['aria-label']
        get_score = get_score.replace("評等：","")
        get_score = get_score.replace(" (最高：5)，","")
        print(get_score)

        #評論
        try:
            #直接用try抓評論全文
            what_comment = i.select_one('div[class="Jtu6Td"] span[jscontroller="MZnM8e"] span[class="review-full-text"]').text
        except:
            try:
                #如果沒有全文，可能是文不夠長，所以到這個路徑抓
                what_comment =i.select_one('div[class="Jtu6Td"] span[jscontroller="MZnM8e"] span[tabindex="-1"]').text
                try:
                    #因為比較短的評論會跟'造訪時間'等固定格式擺在一起
                    #所以再try一次抓固定格式，如果有的話就把它從字串中移除
                    more_comment =i.select_one('div[class="Jtu6Td"] span[jscontroller="MZnM8e"] span[tabindex="-1"] div[class="k8MTF"]').text
                    what_comment = what_comment.replace(more_comment,'')
                    #如果移除了之後變成空字串，那就直接變null
                    if what_comment == "":
                        what_comment = "null"
                except:
                    pass
            except:
                #如果沒有評論就打null
                what_comment = "null"
                pass
        print(what_comment)
        print("="*50)

        if what_comment == "null":
            continue
        else:
            if "(由 Google 提供翻譯) " in what_comment:
                new_what_comment = what_comment.replace("(由 Google 提供翻譯) ","")
                strDelimiter = '(原始評論)'
                new_what_comment = new_what_comment.split(strDelimiter)[0]
                what_comment = new_what_comment
            else:
                pass
            #用字典記錄起來
            comment_dic = {"評論時間":time_comment,"評論內容":what_comment,"評分":get_score}
            #再放進list裡
            comment_list.append(comment_dic)
    return comment_list

In [ ]:
#用結巴斷句，存入一個list
def jieba_cut(comment_list):
    corpus=[]
    for x in comment_list:    
        cut_word = x['評論內容']
        corpus += jieba.lcut(cut_word,cut_all=False)
    
    stop_word_list = []
    with open("./stopwordlist.txt",mode = "r",encoding = "utf-8") as file:
        for word in file:
            stop_word_list.append(word.strip())
        stop_word_list.append('，')
        stop_word_list.append(' ')
        stop_word_list.append('地方')
    
    # 統計分詞出現次數
    dictionary = Counter(corpus)

    # 移除停用詞
    [dictionary.pop(x, None) for x in stop_word_list] # 存字典裡刪除停用詞

    word_dic = dict(dictionary)

    return word_dic

In [ ]:
def make_cloud(word_dic,place_name):
    #建立一個放文字雲圖的資料夾
    folderPath = 'wordcloud_img'
    if not os.path.exists(folderPath):
        os.makedirs(folderPath)

    fontpath = "c:/test-wordcloud/msjh.ttc"  # 字型檔

    pngfile = "./cloud.jpg"  # 剛才下載存的底圖

    alice_mask = numpy.array(Image.open(pngfile))

    wc = WordCloud(background_color="white", mask=alice_mask, contour_width=3, contour_color='steelblue', font_path= fontpath).generate_from_frequencies(word_dic)
    plt.figure()
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.show()

    wc.to_file(f'./wordcloud_img/WordCloud_{place_name}.png')   #將圖存到資料夾內

In [ ]:
#把group_place_list內的每個小組抓出來分別搜索裡面的評論，然後做出文字雲，最後再統整所有景點的所有評論並送出

def get_place_comment(all_group_place,get_group): 
    time_sleep = random.randint(5,15)
    
    get_group = get_group - 1
    
    collect_every_comment_place = []

    for item in all_group_place[get_group]:    #每一圈都是一個地點
        every_comment_place = {}
        
        every_comment_place["景點名稱"] = item["景點名稱"]   #要做成 [{'景點名稱':'中正紀念堂','全部評論':[{評論1},{評論2}...]}

        #抓出存有連結的key-value
        item_href = item["評論連結"]
        print(item_href)

        #如果沒有連結就跳過
        try:
            if item_href != "null":
                #有連結就連上
                try:
                    driver.get(item_href)

                    #睡一下等網頁跑
                    sleep(time_sleep)

                except:
                    driver.refresh()
                    sleep(time_sleep)

                finally:
                    driver.refresh()

                sleep(10)

                #連上網頁之後開始抓評論
                #先將評論調成最新
                set_newest()

                #開始滾動
                scroll()

                #滾完之後抓出所有評論
                #這是一個地點的所有評論
                comment_list = get_all_comment()    #return出來的comment_list是一個由dict構成的list，裡面每個dict都是一則評論 [{},{}...]

                every_comment_place["全部評論"] = comment_list

                #將所有評論拿去斷句
                word_dic = jieba_cut(comment_list)

                #將斷句結果拿去做成文字雲
                make_cloud(word_dic,item["景點名稱"])

            else:
                every_comment_place["全部評論"] = ["null"]
        except:
            pass

        
        collect_every_comment_place.append(every_comment_place)
        #將every_comment_place這個包含地點跟所有評論的dict放入collect_every_comment_place這個list

        print(every_comment_place)

        print("="*50)

        sleep(time_sleep)
    
    print(collect_every_comment_place)
    #小組之中的每個景點的評論都爬完收集成dict再放入list之後，把list丟出來
    
    return collect_every_comment_place    #由dict組成的list


In [ ]:
# ======================= Part 2 總執行 ===========================

In [ ]:
# 使用抓出特定小組的func
#再從 all_group_place[get_group] 裡面抓出 "評論連結"，連上連結  >>>  評論調成最新  >>>  滾動頁面  >>>  用bs抓出全部評論  >>>  存起來  >>>  斷句  >>>  文字雲

final_comment_data = []

In [ ]:
final_comment_data = save_final_comment_data
print(final_comment_data)

In [ ]:
#繼續使用抓出特定小組的func
#控制要爬哪一組  >>>  將每組爬完新增的資訊再裝起來

get_group = get_what_group()
collect_every_comment_place = get_place_comment(all_group_place,get_group)    #這步會得到所有景點的文字雲跟這個形式的全部景點跟評論[{'景點名稱':'中正紀念堂','全部評論':[{評論1},{評論2}...]}]
#一組一組做，最終得到

final_comment_data.append(collect_every_comment_place)
# final_comment_data是 [[{"景點名稱":"中正紀念堂","全部評論":[{評論1},{評論2}...]}],[{},{}...]...]

In [ ]:
print(final_comment_data)

In [ ]:
#把每個小組打散
new_final_comment_data = []
for item in final_comment_data:
    for inner_item in item:
        new_final_comment_data.append(inner_item)
print(new_final_comment_data)

In [ ]:
#將最後集合完的list匯出成excel

#把抓出來的東西寫入excel
wb2 = openpyxl.Workbook()

#把 final_comment_data 內的東西撈出來
for first in new_final_comment_data:    # first代表每一組景點的list [{"景點名稱":"中正紀念堂","全部評論":[{評論1},{評論2}...]}]
    #先建立一頁新的sheet
    sheet_name = first["景點名稱"]
    new_sheet = wb2.create_sheet(f'{sheet_name}')
    
    #設定欄位
    wb2.active = new_sheet
    worksheet = wb2.active
    worksheet.append(["評論時間","評分"])
    
wb2.save("./place_commentTime_score_0607.xlsx")
wb2.close()

In [ ]:
#先讀取相同名稱的sheet
# 加载现有的Excel文件

from openpyxl import load_workbook
workbook = load_workbook("./place_commentTime_score_0607.xlsx")

for first in new_final_comment_data:
    #先用for迴圈抓出景點名稱
    name = first['景點名稱']
    if name == '台北市臨江街觀光夜市｜通化街夜市 Taiwan Taipei Night Market':
        name = '通化街夜市'
    print(name)
    
    # 通过名称打开工作表
    worksheet = workbook[f"{name}"]
    
    #再抓出全部評論
    try:
        take_comments = first['全部評論']    #這是個list
    except:
        continue    #沒有評論的項目直接跳過    
    
    if first['全部評論'] == ['null']:    #{'景點名稱': '信義區', '全部評論': ['null']}
        continue
    
    #所以再用for迴圈全部抓出來
    for second in take_comments:    #second是 {'評論時間': '2 小時前', '評論內容': '六四的活動', '評分': '5.0'} 這樣的東西
        time_comment = second['評論時間']
        score = second['評分']
        
        #放進sheet內
        worksheet.append([time_comment,float(score)])
    
    #存檔
    workbook.save("./place_commentTime_score_0607.xlsx")
workbook.close()

